<a href="https://colab.research.google.com/github/yoojinko24/Data-Projects/blob/main/9%EC%A1%B0_%EB%B8%94%EB%A3%A8%EC%98%A4%EC%85%98_%EC%A7%80%EC%97%AD_%EC%B6%94%EC%B2%9C_%EC%84%9C%EB%B9%84%EC%8A%A4_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8_%EC%99%84%EC%84%B1%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 폰트 설치

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [ ]:
# 구글 드라이브와 연동

from google.colab import drive
drive.mount('/content/gdrive')
directory_path = '/content/gdrive/My Drive/Colab Notebooks/23-1 초급파데분 팀플/최종 제출 파일 (데이터셋, ppt, 리포트, 동영상, 리드미))/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

# A. 데이터처리 라이브러리 불러오기

import numpy as np
import pandas as pd
import csv

################################################################################


# B. 시각화 라이브러리 불러오기

import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

################################################################################


# C. 데이터 전처리

# C-1. 업종 필터링을 위한 업종 데이터 로드 (특별한 전처리 없이 그대로 불러와서 사용)
business = pd.read_csv(directory_path+'소상공인시장진흥공단_상가(상권)정보 업종코드_20230228.csv',encoding='cp949')


#-------------------------------------------------------------------------------

# C-2. 생활인구 데이터 전처리

  # index_col은 데이터 안의 첫번째 열이 인덱스로 되는 것을 방지하기 위해 추가
df_pop=pd.read_csv(directory_path + '행정동 단위 서울 생활인구.csv',encoding='utf-8', index_col=False)

  # 사용하지 않는 나잇대별 생활인구 열 제거
df_pop=df_pop.iloc[:, :4]

  # 행정동 코드 리스트 뽑아서 정리
cod=list(set(df_pop.iloc[:, 2]))
cod.sort()

  # 행정동 별 총생활인구수(시간대 총합) 데이터프레임 생성 (인덱스를 행정동 코드로 설정)
df_peo=pd.DataFrame([], columns=['총생활인구'], index=cod)

  # groupby를 이용해 df_pop에서 행정동코드가 일치하는 행들에서 총생활인구수를 모두 더해, 총생활인구수에 더함
df_peo = df_pop.groupby('행정동코드')['총생활인구수'].sum().reset_index()
pd.options.display.float_format = '{:.5f}'.format #형식을 맞추는 코드

  # 계산한 총생활인구수는 올해 2월 1일~28일 사이의 생활인구를 모두 더한 값이므로 28로 나누고, 행정동코드 별 1일 평균 생활인구값을 도출
df_peo['총생활인구수'] = df_peo['총생활인구수'] / 28

  # 행정동코드와 동이 연결된 데이터 불러오기
df_dong=pd.read_csv(directory_path + 'KIKmix.20230224(말소코드포함).csv',encoding='cp949')

  # 법정동 코드 열 제거
df_dong=df_dong.iloc[:, :4]

  # 시도명이 서울특별시로 되어있지 않은 값은 모두 삭제, 그리고 NaN값이 포함되어 있는 행도 모두 삭제
idx = df_dong[df_dong['시도명']!='서울특별시'].index
df_dong = df_dong.drop(idx)
df_dong=df_dong.dropna(axis=0)

df_dong=df_dong.drop_duplicates(['행정동코드']) #행정동코드가 같은 데이터들이 있다. 그래서 중복되는 값들을 지워준다.
df_dong['행정동코드'] = df_dong['행정동코드'] / 100 #생활인구 데이터의 행정동코드와 이 데이터의 행정동코드에 차이가 있어서 넣은 코드(행정동코드 데이터에서 100을 나눠주면 동일해짐)
df_dong['행정동코드'] = df_dong['행정동코드'].astype('int64') #데이터타입을 맞춰줌
df_dongpeo=pd.merge(df_dong, df_peo, on='행정동코드') #최종 우리가 생활인구수로 사용할 데이터는 df_dongpeo 데이터이다. 행정동 코드가 일치하는 곳에 두 데이터프레임을 합쳐줌


#-------------------------------------------------------------------------------

# C-3. 동종업계 데이터 전처리

  # 동종업계 데이터 로드 (특별한 전처리 없이 그대로 불러와서 사용)
df_bus=pd.read_csv(directory_path + '소상공인시장진흥공단_상가(상권)정보_서울_202303.csv',encoding='utf-8')


#-------------------------------------------------------------------------------

# C-4. 잠재소비자 데이터 전처리과정

  # 업종별 소비금액 데이터 불러오기
customer_data=pd.read_csv(directory_path+'서울시 우리마을가게 상권분석서비스(상권-소득소비).csv', encoding='cp949')

  # 상권 영역 동별로 분류하는 데이터 불러오기
sale_code_data=pd.read_csv(directory_path+'서울시 우리마을가게 상권분석서비스(상권영역).csv',encoding='cp949')

  # 상권 코드 기준으로 두 데이터 합치기(상권-행정동코드 이어주기 위함)
join = pd.merge(customer_data, sale_code_data, on="상권_코드")
join2 = pd.merge(join, df_dong, left_on=["행정동_코드"], right_on=["행정동코드"]) # 행정동코드 데이터 불러와서 행정동 달아주기

pd.options.display.float_format = '{:.2f}'.format   # 수치가 지수 e로 저장돼서 float로 형 변환


  # 필요없는 열 제거
join2.drop(["상권_구분_코드_x", "상권_구분_코드_명_x", "월_평균_소득_금액", "소득_구간_코드", "기준_년월_코드",
            "상권_구분_코드_y", "상권_구분_코드_명_y", "상권_코드_명_y", "엑스좌표_값", "와이좌표_값",
            "형태정보", "행정동_코드", "시도명"], axis=1, inplace=True)

  # 같은 읍면동명 가진 행끼리 업종 컬럼별로 합해줌
df_customer=join2.groupby("읍면동명")[['지출_총금액', '식료품_지출_총금액',
       '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액']].sum()

  # 컬럼명 변경
df_customer.rename(columns={'지출_총금액':'총 금액',
                     '식료품_지출_총금액':'식료품',
                     '의류_신발_지출_총금액':'의류',
                     '생활용품_지출_총금액':'생활용품',
                     '의료비_지출_총금액':'의료',
                     '교통_지출_총금액':'교통',
                     '여가_지출_총금액':'여가',
                     '문화_지출_총금액':'문화',
                     '교육_지출_총금액':'교육',
                     '유흥_지출_총금액':'유흥'}, inplace=True)

df_customer=df_customer.replace(0, np.nan)


#-------------------------------------------------------------------------------

# C-5. 폐업률 데이터전처리 과정

  # 폐업률 데이터 불러오기
closed=pd.read_csv(directory_path+'서울시_우리마을가게_상권분석서비스(신_상권_점포)_2021년.csv',encoding='cp949')

  # '상권_코드' 컬럼을 기준으로 합치기
final_closed = pd.merge(sale_code_data, closed, on='상권_코드')

  # 필요없는 컬럼 제거
columns_to_drop = ["기준_분기_코드", "서비스_업종_코드", "서비스_업종_코드_명",
                    "유사_업종_점포_수", "개업_율", "개업_점포_수", "프랜차이즈_점포_수",'기준_년월_코드', '폐업_률',
                   "상권_구분_코드_x", "상권_구분_코드_명_x", "상권_코드_명_x", "상권_구분_코드_y", "상권_구분_코드_명_y", "상권_코드_명_y", "엑스좌표_값", "와이좌표_값", "시군구_코드", "형태정보"]
final_closed.drop(columns_to_drop, axis=1, inplace=True)

  # 이름 변경
final_closed.rename(columns={'기준_년_코드':'기준년도',
                       '행정동_코드':'행정동 코드',
                       '상권_코드':'상권 코드',
                       '폐업_점포_수':'폐업 점포 수',
                       '점포_수': '점포 수'}, inplace=True)
  # 행정동별 폐업률 구하기
final_closed=final_closed.groupby('행정동 코드')[['폐업 점포 수', '점포 수']].sum()   # 행정동 별로 폐업 점포수와 전체 점포 수 각각 합하기
final_closed['폐업률']=(final_closed['폐업 점포 수']/final_closed['점포 수'])*100   # 폐업점포수 총합/전체 점포수 총합*100으로 폐업률 구함
dong_final_closed=pd.merge(final_closed, df_dong, left_on='행정동 코드', right_on="행정동코드")   # 행정동코드 기준으로 데이터 합침
final_closed_drop=dong_final_closed.groupby('읍면동명')['폐업률'].mean()     # 중복되는 행 평균내서 통일


#-------------------------------------------------------------------------------

# C-6. 임대료 데이터 전처리

  # 임대료 데이터 불러오기
rent_fee = pd.read_csv(directory_path+'한국부동산원_상업용부동산 임대동향조사_임대정보_분기별 지역별 임대료(소규모상가).csv',encoding='cp949')

  # '행정구 코드명' 컬럼명을 '시군구명'으로 변경
rent_fee.rename(columns={'행정구 코드명':'시군구명'},inplace=True)

  # '지역', '2022_1분기', '2022_2분기' 컬럼 삭제
rent_fee.drop(['지역','2022_1분기','2022_2분기'],axis=1,inplace=True)

  # '시군구명'으로 그룹화하여 '2022_상반기_평균' 평균 계산
rent_fee = rent_fee.groupby(['시군구명'])['2022_상반기_평균'].agg('mean').reset_index()

  # '시군구명'을 기준으로 'df_dong' 데이터프레임과 'outer' 방식으로 병합
merge_fee = pd.merge(df_dong,rent_fee, how='outer',on='시군구명')

  # 인덱스 초기화 후 '읍면동명'을 인덱스로 지정
merge_fee = merge_fee.reset_index().set_index(['읍면동명'])

  # 'index'와 '시도명' 컬럼 삭제
merge_fee.drop(['index','시도명'],axis=1,inplace=True)

  # 중복 제거
merge_fee.drop_duplicates(inplace=True)

  # 동일 행정동명 처리
merge_fee_drop=merge_fee.groupby(merge_fee.index)['2022_상반기_평균'].mean()


#-------------------------------------------------------------------------------


# C-7. 매출액 데이터 전처리

  # 매출액 데이터 프레임 생성
  # 1-1. 상권분석서비스 데이터를 불러와 데이터프레임으로 변환
sales = pd.read_csv(directory_path+'서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2021년.csv', encoding='cp949')

  # 1-2. 필요한 열만 선택하여 데이터프레임 생성
sales = sales[['상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '주중_매출_금액', '주말_매출_금액']]

  # 1-3. '주중매출_금액'과 '주말_매출_금액' 컬럼을 더한 '총_매출_금액' 컬럼 생성
sales['총_매출_금액'] = sales['주중_매출_금액'] + sales['주말_매출_금액']
sales = sales.set_index('상권_코드').sort_index(ascending=True)



  # 행정동 별 매출 데이터 프레임 생성 처리
  # 2-1. sale_code_data에서 '상권_코드' 기준으로 인덱스 지정
reg_sales = sale_code_data.set_index('상권_코드')

  # 2-2. sales와 reg_sales를 '상권_코드'를 기준으로 병합 merged_for_sales 생성
merged_for_sales = pd.merge(sales, reg_sales[['행정동_코드']],
                            left_index=True, right_index=True, how='inner')

  # 2-3. 인덱스 초기화 후 '행정동_코드'를 기준으로 인덱스 지정
merged_for_sales = merged_for_sales.reset_index().set_index(['행정동_코드'])

  # 2-4. '행정동_코드'를 기준으로 그룹화하여 '총_매출_금액' 평균 계산
mean_sales = merged_for_sales.groupby('행정동_코드')['총_매출_금액'].mean()

  # 2-5. 중복 제거한 데이터프레임 생성
final_sales = pd.DataFrame({'총_매출_금액_평균': mean_sales})

  # 2-6. '행정동_코드'를 인덱스 이름으로 변경 및 인덱스 초기화
final_sales.index.name = '행정동_코드'
final_sales.reset_index(inplace=True)


  # 지수 표현 없이 소수점을 아래쪽으로 2자리까지 출력하도록 설정
pd.options.display.float_format = '{:.1f}'.format

  # df_dong 데이터프레임과 병합해, 최종 데이터프레임 생성
final_sales = pd.merge(final_sales, df_dong, left_on=['행정동_코드'], right_on=['행정동코드'], how='left')
final_sales = final_sales.set_index(['읍면동명'])


  # 중복된 행정동은 두 값을 평균내어 하나로 통일시킴
final_sales_drop=final_sales.groupby(final_sales.index)['총_매출_금액_평균'].mean()



################################################################################

# D. 프로그램 실행을 위한 함수 프로그래밍


# D-1. 창업 희망 업종 대분류/중분류 입력받는 함수
def business_filter():

    # 1. 대분류명 목록 출력
    print('📝 업종 리스트','>>',np.unique(business['대분류명']))
    print()

    # 2. 대분류 입력 받기
    category = input('🧡 어떤 분야의 창업을 원하시나요? 리스트에서 그대로 입력해주세요:')

    # 3. 중분류명 목록 출력
    category_list = np.unique(business[business['대분류명'] == category]['중분류명'])

    # 2-1. 입력한 대분류에 중분류 데이터가 없으면 대분류를 잘못 입력했다고 판단
    while not np.any(category_list):
        print('해당하는 업종 대분류가 없습니다.\n')
        category = input('조회할 업종의 대분류명을 다시 입력해주세요: \n')
        category_list = np.unique(business[business['대분류명'] == category]['중분류명'])

    print()
    print('\n📝 세부 업종 리스트', '>>',category_list)
    print()


    # 4. 중분류 입력 받기
    final_category = input('🧡 창업하고 싶은 {}의 세부 업종을 그대로 입력해주세요:'.format(category))

    # 4-1. 입력한 중분류가 해당 대분류명이 가지고 있는 리스트에 없으면 잘못 입력했다고 판단
    while final_category not in category_list:
        final_category = input('다시 입력해주세요: ')

    # 4-2. 중분류 코드 저장
    category_code=np.unique(business[business['중분류명']==final_category]['중분류코드'])[0]

    # 5. 입력한 값(중분류명)을 final_category로 받아두고, 다음 블루오션 필터링에서 검색어로 활용한다.

    # 6. 프로그램 결과 출력
    print('\n...\n...','\n 서울시내 {} 블루오션 찾기를 시작합니다.'.format(final_category))

    return final_category, category, category_code


#-------------------------------------------------------------------------------

# D-2. 블루오션 필터링 함수 bluefilter 정의

def bluefilter(category, df_bus, df_dongpeo, final_category_kor): #전처리한 데이터와, 입력받은 업종(category)를 받아 사용합니다

  # 필요한 데이터 정의

  # 1. filtered_data : 특정 업종을 가진 상권 데이터를 분리해낸 데이터
  # 입력한 업종코드에 해당하는 상권을 filtered_data에 넣어준다. 예를 들어 한식(코드 I201)을 입력했을 경우, 한식업종 가게만 담은 데이터가 됨
  filtered_data = df_bus[df_bus['상권업종중분류코드'] == category].copy()

  # dong_codes는 특정 업종 상권을 가진 행정동의 코드를 리스트로 넣은 것.
  dong_codes = list(set(filtered_data['행정동코드']))

  # 2. df_filtered_dongpeo : 생활인구 데이터에서 특정 업종 상권을 가진 행정동에 해당하는 생활인구 데이터만을 뽑기 위한 데이터프레임
  df_filtered_dongpeo=pd.DataFrame()
  for i in range(len(dong_codes)):     # df_filtered_dongpeo 데이터프레임을 만드는 과정-행정동코드가 dong_codes에 존재할 경우, df_dongpeo의 값을 넣어준다.
    for j in range(len(df_dongpeo)):
      if df_dongpeo['행정동코드'][j] == dong_codes[i]:
            df_filtered_dongpeo = df_filtered_dongpeo.append(df_dongpeo.iloc[j])
  df_filtered_dongpeo.reset_index(drop=True, inplace=True)
  df_filtered_dongpeo = df_filtered_dongpeo.reindex(range(len(df_filtered_dongpeo)))

  # 최종 나오는 데이터는 df_blue데이터. 여기에 행정동 코드, 이름, 동종업계, 생활인구 모두 넣어 블루오션 필터링에 사용
  df_blue=pd.DataFrame()

  # df_blue의 컬럼들을 만들어주고, 행정동코드명은 미리 넣어준다
  df_blue['행정동코드명']=dong_codes
  df_blue['총생활인구'] = 0
  df_blue['동종업계 수']= 0
  df_blue['행정동명']=np.nan

  # df_blue에 총생활인구, 동종업계 수를 넣어주는 과정
  for i in range(len(df_blue)):
    for j in range(len(df_filtered_dongpeo)):
      if df_filtered_dongpeo['행정동코드'][j] == df_blue['행정동코드명'][i]:
              df_blue['총생활인구'][i] = df_filtered_dongpeo['총생활인구수'][j]
  for i in range(len(df_blue)):
    df_blue['동종업계 수'][i]=len(filtered_data[filtered_data['행정동코드'] == df_blue['행정동코드명'][i]])

  # df_blue에 행정동 코드가 있고 행정동명이없으므로 넣어줌
  for i in range(len(df_filtered_dongpeo)):
    for j in range(len(df_blue)):
      if df_blue['행정동코드명'][j]==df_filtered_dongpeo['행정동코드'][i]:
        df_blue['행정동명'][j]=df_filtered_dongpeo['읍면동명'][i]

  # 지수컬럼 만드는 코드. 지수 = 총생활인구/동종업계 수, 지수가 클수록 더 블루오션
  df_blue['지수']=df_blue['총생활인구']/df_blue['동종업계 수']
  df_sorted_blue = df_blue.sort_values(by='지수', ascending=False)
  print('\n\n***❤️'+ str(final_category_kor) + ' 업종의 동종업계 대비 인구가 많은 행정동 상위 30개는 다음과 같습니다❤️***')
  print(list(df_sorted_blue['행정동명'].head(30)))
  print('\n\n\n')

  return df_sorted_blue.head(30)


#-------------------------------------------------------------------------------

# D-3. 가중치 적용 함수
def score_weight():
  print('🧡 아래 항목들의 번호를 중요하다고 생각되는 순서대로 쉼표(,)로 구분하여 나열해주세요.\n')
  print('1. 잠재소비자 수       2. 폐업률       3. 임대료       4.매출\n')
  print('예시: 4,2,3,1')
  weight=input('입력: ')            # 순위 입력받아 weight에 저장
  print()

  weight_list=[]

  if ',' not in weight:               # 쉼표로 구분 안했을 경우 다시 입력받음
    print('쉼표를 포함하여 입력해주세요.\n')
    return score_weight()

  else:
    weight_str=weight.split(',')      # 쉼표 기준 split해서 weight_str에 저장
    for i in range(len(weight_str)):
      if weight_str[i].isdigit():     # weight_str에 저장된 문자열들이 전부 숫자라면
        weight_list=np.array(weight_str, dtype=int)      # 정수형으로 변환하여 weight_list에 저장
      else:
        print('숫자를 입력해주세요. \n')    # weight_str이 숫자가 아니라면 다시 입력받음
        return score_weight()

      if weight_list[i]<1 or weight_list[i]>4:        # weight_list의 요소가 1부터 4까지의 숫자가 아니라면 다시 입력받음
        print('1부터 4까지의 숫자를 입력해주세요.\n')
        return score_weight()

  return weight_list


#-------------------------------------------------------------------------------

# D-4. 점수화 함수
def score(filter,upjong,weight):      # 매개변수는 블루오션 필터링 거친 데이터/ 업종명(대분류) / 가중치
  dongs=filter['행정동명']    # 위 블루오션에서 필터링된 행정동들 dongs에 저장

  db_df=pd.DataFrame(index=dongs, columns=['잠재소비자','폐업률','임대료', '매출'])   # 각 데이터 저장할 데이터프레임 생성

  # 업종이 소매업이면 잠재소비자의 생활용품 컬럼 저장
  if upjong == '소매업':
    db_df['잠재소비자']=df_customer['생활용품']
    db_df['잠재소비자']=db_df['잠재소비자'].fillna(db_df['잠재소비자'].mean())    # 결측치는 각 컬럼의 평균값

  # 숙박업이나 여가관련 서비스업이면 잠재소비자의 여가 컬럼 저장
  elif upjong == '숙박업' or upjong=='예술, 스포츠 및 여가관련 서비스업':
    db_df['잠재소비자']=df_customer['여가']
    db_df['잠재소비자']=db_df['잠재소비자'].fillna(db_df['잠재소비자'].mean())

  # 업종이 음식점이면 잠재소비자의 식료품 컬럼 저장
  elif upjong == '음식점업':
    db_df['잠재소비자']=df_customer['식료품']
    db_df['잠재소비자']=db_df['잠재소비자'].fillna(db_df['잠재소비자'].mean())

  elif upjong == '교육 서비스업':
    db_df['잠재소비자']=df_customer['교육']
    db_df['잠재소비자']=db_df['잠재소비자'].fillna(db_df['잠재소비자'].mean())

  elif upjong == '보건의료업':
    db_df['잠재소비자']=df_customer['의료']
    db_df['잠재소비자']=db_df['잠재소비자'].fillna(db_df['잠재소비자'].mean())

  # 이외에 잠재소비자 컬럼으로 분류할 수 없는 업종들은 총 소비자 기준으로 구함
  else:
    db_df['잠재소비자']=df_customer['총 금액']
    db_df['잠재소비자']=db_df['잠재소비자'].fillna(db_df['잠재소비자'].mean())

  db_df['임대료']=merge_fee_drop      # 임대료 데이터 저장
  db_df['임대료']=db_df['임대료'].fillna(np.mean(db_df['임대료']))

  db_df['폐업률']=final_closed_drop   # 폐업률 데이터 저장
  db_df['폐업률']=db_df['폐업률'].fillna(np.mean(db_df['폐업률']))

  db_df['매출']=final_sales_drop      # 매출 데이터 저장
  db_df['매출']=db_df['매출'].fillna(np.mean(db_df['매출']))

  df_sum=0
  for i in range(4):      # 점수화: 가중치는 1순위는 40%, 2순위는 30%, 3순위는 20%, 4순위는 10%
    if weight[i]-1==1 or weight[i]-1==2:        # 폐업률, 임대료는 수가 작을수록 높은 rank를 갖도록 함
      df_sum+=db_df.iloc[:, weight[i]-1].rank(ascending=True)*(40-(10*i))     # 각 컬럼에 rank를 매기고 가중치를 곱해준 후 그 총합을 구함
    else:                   # 잠재소비자, 매출은 수가 클수록 높은 rank를 갖도록 함 (ascending=False)
      df_sum+=db_df.iloc[:, weight[i]-1].rank(ascending=False)*(40-(10*i))

  df_sum=df_sum/100

  samescored=df_sum.duplicated(keep=False)          # 동점값 찾기
  samescoresindex=df_sum[samescored].index          # 어느 동이 동점을 갖는지 알아냄

  filterdong=filter.set_index('행정동명')

  # 점수화함수 기준으로 동점인 값도 약간의 차등을 두기 위해 블루오션에서 사용했던 지수 비교
  ssstandard=filterdong.loc[samescoresindex]['지수'].rank()     # 동점을 갖는 동들의 블루오션 지수에 rank를 매김
  ssstandard=ssstandard/20              # 블루오션 지수는 동점인 동들 사이에만 순위에 영향을 끼치도록 수를 가공
  sstandard=ssstandard.reindex(df_sum.index)
  filled_standard=sstandard.fillna(0)       # NaN 값 빼면 결과도 Nan으로 나와서 전부 0으로 채워줌

  df_sum=df_sum-filled_standard           # 각 점수에서 블루오션 점수를 뺌-동점인 행 간 차등이 생김

  return df_sum.sort_values().head(3), db_df    # 순위가 가장 높은 상위 3개 값 출력


################################################################################


# E. 프로그램 실행 코드

final_category_kor, category, category_code =business_filter()
filtered=bluefilter(category_code, df_bus, df_dongpeo, final_category_kor)
w=score_weight()
result_dongs=score(filtered, category, w)     #result_dongs[0]은 상위 3개 동, result_dongs[1]은 전체 동 데이터
i=1
for dong in result_dongs[0].index:
  print(i,'순위 동은', dong+'입니다.')      # 점수화함수 결과 출력
  i=i+1


################################################################################


# F. 추가정보 제공(정보 시각화)

while True:
  ans = input('\n❤️ 추천 3개동에 대한 위치와 추가 정보 제공을 원한다면 1을, 아니면 2를 입력해주세요: ')
  if ans=='1':

    # 시각화에 쓸 데이터프레임 생성
    first_dong=pd.DataFrame(result_dongs[1].loc[result_dongs[0].index[0]])
    second_dong=pd.DataFrame(result_dongs[1].loc[result_dongs[0].index[1]])
    third_dong=pd.DataFrame(result_dongs[1].loc[result_dongs[0].index[2]])

    final_dong = pd.concat([first_dong, second_dong, third_dong],axis=1).transpose()


    # F-1. 상위 3개동 지도에 마킹

    # 점포 수+위도/경도 데이터 불러오기
    df_b = pd.read_csv(directory_path+'소상공인시장진흥공단_상가(상권)정보_서울_202303.csv',encoding='utf-8')

    # 추천 3개동에 대한 위도/경도 데이터
    sub_df = df_b.loc[df_b['행정동명'].isin([final_dong.index[0], final_dong.index[1], final_dong.index[2]])]

    # 지도의 시작점 설정
    latitude = 37.559819
    longitude = 126.963895

    print("\n\n❤️ 지도 줌인, 줌아웃으로 추천 3개 동의 위치와 상점 수를 확인하세요.\n")

    # 지도 그리기
    m = folium.Map(
        location=[latitude, longitude],
        zoom_start=12
    )

    # spot으로 데이터 추출
    spot = sub_df[['위도', '경도']]

    marker_cluster = MarkerCluster().add_to(m)

    for lat, long in zip(spot['위도'], spot['경도']):
        # foluim.Marker로 마커 생성
        folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
    map_html = m._repr_html_() # 지도를 화면에 표시할 수 있도록, m을 html언어로 저장
    display(HTML(map_html)) # html을 이용해 지도를 화면에서 볼 수 있음

#-------------------------------------------------------------------------------

    # F-2. 상위 3개동 블루오션 지수 비교

    print("\n\n❤️ 추천 3개 동의 총생활인구와 동종업계 수를 비교합니다.\n")

    dong_compare = pd.DataFrame(columns=filtered.columns)
    threedong = result_dongs[0]
    df_threedong=pd.DataFrame(threedong)

    # threedong에 있는 각 행정동명을 순회하면서 처리
    for dong in threedong.index:
        for i in range(len(filtered)):
            if filtered.iloc[i, 3] == dong:
                matching_rows = filtered.iloc[[i]]
                dong_compare = dong_compare.append(matching_rows)
    dong_compare = dong_compare.reset_index(drop=True)

    bar_width = 0.5
    total_peo=np.array(dong_compare['총생활인구'])
    same_bus=np.array(dong_compare['동종업계 수'])
    dong_name=dong_compare.iloc[:, 3]

    plt.rc('font', family='NanumBarunGothic')
    plt.title('생활인구&동종업계 비교 그래프')

    # 생활인구 그래프 바, 값을 그대로 그래프화하면 동종업계 수가 거의 보이지 않기 때문에, 적절한 값으로 생활인구를 나누어줌
    b1 = plt.bar(dong_compare.index, total_peo/10000, color='#9BCA3E', label='생활인구/10^4')

    # 동종업계 수 그래프 바
    b2 = plt.bar(dong_compare.index, same_bus, color='#66B5D7', label='동종업계 수')

    plt.xlabel('행정동명', size = 13)
    plt.xticks(dong_compare.index, labels=dong_name)
    plt.legend()
    plt.show()


#-------------------------------------------------------------------------------

    # F-3. 상위 3개동 상세 비교를 위한 막대그래프 출력

    print("\n\n❤️ 추천 3개 동의 잠재소비자 수, 폐업률, 임대료, 매출을 비교합니다.\n")



    # 추천 3개 동과 상위 30개 동 평균치 하나의 데이터프레임화
    mean_values = result_dongs[1].mean().round(1)
    mean_closure_rate = mean_values['폐업률']
    mean_rent = mean_values['임대료']
    mean_sales = mean_values['매출']
    mean_potential_customer=mean_values['잠재소비자']

    final_dong.loc['평균'] = [mean_potential_customer, mean_closure_rate, mean_rent, mean_sales]


    # y축 값 저장
    ys1 = final_dong['잠재소비자'].to_list()
    ys2 = final_dong['폐업률'].to_list()
    ys3 = final_dong['임대료'].to_list()
    ys4 = final_dong['매출'].to_list()

    # 그래프 배열 생성
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))

    # 각 subplot에 그래프 그리기
    axs[0, 0].bar(final_dong.index, ys1, width=0.6, color= ['skyblue', 'skyblue', 'skyblue', 'grey'])
    axs[0, 0].set_title('상위 3개 동의 잠재소비자 수 비교')
    axs[0, 1].bar(final_dong.index, ys2, width=0.6, color=['pink', 'pink', 'pink', 'grey'])
    axs[0, 1].set_title('상위 3개 동의 폐업률 비교')
    axs[1, 0].bar(final_dong.index, ys3, width=0.6, color=['pink', 'pink', 'pink', 'grey'])
    axs[1, 0].set_title('상위 3개 동의 임대료 비교')
    axs[1, 1].bar(final_dong.index, ys4, width=0.6, color=['skyblue', 'skyblue', 'skyblue', 'grey'])
    axs[1, 1].set_title('상위 3개 동의 매출 비교')
    # 그래프 크기, x축, y축 이름 설정
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    for ax in axs.flat:
        ax.set_xlabel('상위 3개 동')
        ax.set_ylabel('값')
        ax.tick_params(axis='x', labelrotation=45) # x축 라벨 회전
    plt.show()


################################ 프로그램 끝 ####################################

    break
  elif ans=='2':
    print('끝!')
    break
  else:
    print('\n잘못 입력하셨습니다. 다시 선택해주세요')

📝 업종 리스트 >> ['교육 서비스업' '보건의료업' '부동산업' '사업시설 관리, 사업 지원 및 임대 서비스업' '소매업' '수리 및 개인 서비스업'
 '숙박업' '예술, 스포츠 및 여가관련 서비스업' '음식점업' '전문, 과학 및 기술 서비스업']

🧡 어떤 분야의 창업을 원하시나요? 리스트에서 그대로 입력해주세요:교육 서비스업


📝 세부 업종 리스트 >> ['교육 지원 서비스업' '기타 교육기관' '일반 교육기관']

🧡 창업하고 싶은 교육 서비스업의 세부 업종을 그대로 입력해주세요:일반 교육기관

...
... 
 서울시내 일반 교육기관 블루오션 찾기를 시작합니다.


***❤️일반 교육기관 업종의 동종업계 대비 인구가 많은 행정동 상위 30개는 다음과 같습니다❤️***
['소공동', '수서동', '명동', '화곡제8동', '가양제2동', '황학동', '숭인제2동', '월계3동', '장충동', '가리봉동', '이화동', '면목제5동', '신당동', '동화동', '응봉동', '원효로제1동', '마장동', '한강로동', '영등포동', '회기동', '창신제2동', '을지로동', '삼성동', '도봉제1동', '천호제3동', '면목본동', '중화제2동', '자양제4동', '휘경제1동', '화곡제4동']




🧡 아래 항목들의 번호를 중요하다고 생각되는 순서대로 쉼표(,)로 구분하여 나열해주세요.

1. 잠재소비자 수       2. 폐업률       3. 임대료       4.매출

예시: 4,2,3,1
입력: 1,3,2,4

1 순위 동은 도봉제1동입니다.
2 순위 동은 영등포동입니다.
3 순위 동은 화곡제4동입니다.
